#                                         YOLO模型解读

目前，基于深度学习算法的一系列目标检测算法大致可以分为两大流派：
1.两步走（two-stage）算法：先产生候选区域然后再进行CNN分类(RCNN系列)，
2.一步走（one-stage）算法：直接对输入图像应用算法并输出类别和相应的定位(YOLO系列)
YOLO（you only look once:你只看一次），它的核心思想就是将图片整体分成几个格子，整体识别。利用整张图作为网络的输入，直接在输出层回归 bounding box（边界框） 的位置及其所属的类别。
![图片1](./图片1.jpg)
从图中可以看到，这几种模型YOLO的速度是最快的，YOLO系列由于它只需要识别一次，所以它的识别速度是比较快的。

### YOLO具体实现方式

它将一幅图像分成 SxS 个网格（grid cell），如果某个 object 的中心落在这个网格中，则这个网格就负责预测这个 object。 
每个网格要预测 B 个 bounding box，每个 bounding box 除了要回归自身的位置之外，还要附带预测一个 confidence 值。 
这个 confidence 代表了所预测的 box 中含有 object 的置信度和这个 box 预测的有多准这两重信息，其值是这样计算的：
![图片2](./图片2.png)
其中如果有 object 落在一个 grid cell 里，第一项取 1，否则取 0。 第二项是预测的 bounding box 和实际的 groundtruth 之间的 IoU 值。

每个 bounding box 要预测 (x, y, w, h) 和 confidence 共5个值，每个网格还要预测一个类别信息，记为 C 类。则 SxS个 网格，每个网格要预测 B 个 bounding box 还要预测 C 个 categories。输出就是 S x S x (5*B+C) 的一个 tensor。

注意：class 信息是针对每个网格的，confidence 信息是针对每个 bounding box 的。
举例说明: 在 PASCAL VOC 中，图像输入为 448x448，取 S=7，B=2，一共有20 个类别（C=20），则输出就是 7x7x30 的一个 tensor。
整个网络结构如下图所示： 
![6](./6.png)
在 test 的时候，每个网格预测的 class 信息和 bounding box 预测的 confidence信息相乘，就得到每个 bounding box 的 class-specific confidence score。
等式左边第一项就是每个网格预测的类别信息，第二、三项就是每个 bounding box 预测的 confidence。这个乘积即 encode 了预测的 box 属于某一类的概率，也有该 box 准确度的信息。

得到每个 box 的 class-specific confidence score 以后，设置阈值，滤掉得分低的 boxes，对保留的 boxes 进行 NMS 处理，就得到最终的检测结果。

### YOLO 的实现细节

每个 grid 有 30 维，这 30 维中，8 维是回归 box 的坐标，2 维是 box的 confidence，还有 20 维是类别。 

其中坐标的 x, y 用对应网格的 offset 归一化到 0-1 之间，w, h 用图像的 width 和 height 归一化到 0-1 之间。

在实现中，最主要的就是怎么设计损失函数，让这个三个方面得到很好的平衡。作者简单粗暴的全部采用了 sum-squared error loss 来做这件事。

#### 这种做法存在以下几个问题： 

第一，8维的 localization error 和20维的 classification error 同等重要显然是不合理的； 

第二，如果一个网格中没有 object（一幅图中这种网格很多），那么就会将这些网格中的 box 的 confidence push 到 0，相比于较少的有 object 的网格，这种做法是 overpowering 的，这会导致网络不稳定甚至发散。

#### 解决办法：

更重视8维的坐标预测，给这些损失前面赋予更大的 loss weight, 记为在 pascal VOC 训练中取 5。
对没有 object 的 box 的 confidence loss，赋予小的 loss weight，记为在 pascal VOC 训练中取 0.5。
有 object 的 box 的 confidence loss 和类别的 loss 的 loss weight 正常取 1。
